In [12]:
import matplotlib
import tiktoken
import torch
import torch.nn as nn


GPT_CONFIG_124M = {
"vocab_size": 50257, # Vocabulary size
"context_length": 1024, # Context length
"emb_dim": 768, # Embedding dimension
"n_heads": 12, # Number of attention heads
"n_layers": 12, # Number of layers
"drop_rate": 0.1, # Dropout rate
"qkv_bias": False # Query-Key-Value bias
}

In [32]:
class BaseGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(*[BaseTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = BaseLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits   


In [30]:
class BaseTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x

class BaseLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
    
    def forward(self, x):
        return x


In [14]:
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves a"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  257],
        [6109, 1110, 6622,  257]])


In [34]:
torch.manual_seed(12423)
model = BaseGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 2.2158,  0.6168,  1.6288,  ..., -0.6449, -0.3533,  1.9425],
         [-0.7857, -0.6058,  0.4549,  ...,  1.0483,  0.7585,  1.0739],
         [-2.7803, -0.7292,  0.3345,  ...,  0.8783,  1.3939,  0.9193],
         [ 0.6966,  0.8221, -0.6616,  ..., -0.7818,  0.1671,  1.0633]],

        [[ 1.5109,  0.4453,  1.4775,  ..., -1.3766, -0.2224,  1.9125],
         [-1.4151, -0.0452, -0.4428,  ...,  0.8530,  0.5996,  1.7292],
         [-3.0778, -0.4633, -1.2628,  ...,  0.1009, -0.3442,  0.8746],
         [ 0.3622,  0.6687, -0.9696,  ..., -1.0015,  0.0678,  0.3372]]],
       grad_fn=<UnsafeViewBackward0>)
